In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW

from sklearn.model_selection import train_test_split
import random
import numpy as np

# Clase del conjunto de datos
class TextDataset(Dataset):
    def __init__(self, tokenizer, texts, labels):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
        return encoding['input_ids'].flatten(), encoding['attention_mask'].flatten(), torch.tensor(label)

# Ejemplo de datos
# Ejemplo de textos y etiquetas
texts = [
    "Los avances en la inteligencia artificial están transformando el mundo.",
    "La reforestación es vital para combatir el cambio climático.",
    "Los coches eléctricos son cada vez más populares.",
    "La conservación de los océanos es esencial para la biodiversidad.",
    "La realidad virtual ofrece nuevas oportunidades en el entretenimiento y la educación.",
    "La agricultura sostenible puede ayudar a alimentar a la población mundial.",
    "El aprendizaje automático está revolucionando el análisis de datos.",
    "Los paneles solares son una fuente de energía renovable y sostenible.",
    "La programación es una habilidad clave en el mundo tecnológico actual.",
    "Los ecosistemas forestales juegan un papel crucial en el mantenimiento del equilibrio ambiental.",
    "La inteligencia artificial facilita la predicción de patrones en grandes conjuntos de datos.",
    "La deforestación en la Amazonía afecta la biodiversidad global.",
    "El desarrollo de nuevas aplicaciones móviles mejora nuestra vida diaria.",
    "La protección de especies en peligro de extinción es crucial para el ecosistema.",
    "La tecnología blockchain tiene el potencial de revolucionar el sistema financiero.",
    "La energía eólica es una de las fuentes de energía renovable más prometedoras.",
    "Los avances en robótica están creando nuevas oportunidades en la industria manufacturera.",
    "El calentamiento global es uno de los mayores desafíos ambientales de nuestro tiempo.",
    "Las redes neuronales profundas están transformando la visión por computadora.",
    "La conservación del agua es esencial para la sostenibilidad ambiental.",
    "Los dispositivos vestibles están cambiando la forma en que monitoreamos nuestra salud.",
    "El reciclaje ayuda a reducir la contaminación y conservar los recursos naturales.",
    "Los drones están siendo utilizados en diversas aplicaciones, desde la entrega hasta la fotografía aérea.",
    "La agricultura orgánica promueve prácticas sostenibles y saludables en la producción de alimentos.",
    "El desarrollo de software es una carrera en crecimiento en la era digital.",
    "El deshielo de los glaciares es una señal alarmante del cambio climático.",
    "Los avances en medicina son impulsados significativamente por la tecnología.",
    "La preservación de los arrecifes de coral es vital para la vida marina.",
    "La inteligencia artificial está transformando el sector del transporte.",
    "Las áreas protegidas juegan un papel importante en la conservación de la biodiversidad.",
    "La computación cuántica tiene el potencial de cambiar el panorama tecnológico.",
    "El tratamiento de aguas residuales es fundamental para proteger los ecosistemas acuáticos.",
    "Los satélites artificiales desempeñan un papel crucial en la comunicación global.",
    "El compostaje es un método efectivo para gestionar los residuos orgánicos.",
    "El aprendizaje en línea está democratizando el acceso a la educación.",
    "Los humedales proporcionan hábitats críticos para muchas especies.",
    "Los desarrollos en baterías de iones de litio están impulsando la revolución de los vehículos eléctricos.",
    "La polinización por abejas es esencial para la producción de alimentos.",
    "La realidad aumentada ofrece nuevas formas de interacción digital.",
    "El manejo sostenible de los bosques es clave para combatir la deforestación.",
    "Las redes 5G prometen revolucionar la comunicación inalámbrica.",
    "El cultivo de plantas nativas ayuda a mantener el equilibrio ecológico.",
    "La ciberseguridad se ha vuelto crucial en la era digital.",
    "Los parques nacionales contribuyen a la conservación de la vida silvestre.",
    "Los avances en nanotecnología están abriendo nuevas posibilidades en la medicina.",
    "La jardinería urbana apoya la biodiversidad en las ciudades.",
    "El uso de la inteligencia artificial en la medicina está mejorando los diagnósticos.",
    "Los sistemas de riego eficientes son importantes para la agricultura sostenible.",
    "La realidad virtual está transformando la industria del entretenimiento.",
    "Las medidas de conservación son clave para proteger las especies en peligro.",
    "Los vehículos autónomos cambiarán nuestro modo de transporte.",
    "La plantación de árboles es esencial para restaurar los bosques degradados.",
    "La biotecnología está impactando la industria farmacéutica.",
    "Los humedales proporcionan beneficios importantes para el control de inundaciones.",
    "El desarrollo de nuevas tecnologías de almacenamiento de energía es vital para las energías renovables.",
    "La protección de las abejas es crucial para la polinización y la agricultura.",
    "El big data está revolucionando la toma de decisiones en los negocios.",
    "La conservación de ríos y lagos es vital para mantener los ecosistemas acuáticos.",
    "Los avances en la impresión 3D están transformando la fabricación.",
    "La restauración de hábitats naturales es clave para la biodiversidad.",
    "La inteligencia artificial en el sector bancario está mejorando la eficiencia.",
    "El mantenimiento de los arrecifes de coral es crucial para la vida marina.",
    "Los drones están revolucionando la agricultura con técnicas de monitoreo avanzadas.",
    "Las prácticas de pesca sostenible son importantes para la salud de los océanos.",
    "La automatización en la industria manufacturera está aumentando la producción.",
    "Los esfuerzos de reforestación son críticos para combatir el cambio climático.",
    "Los avances en energía solar están promoviendo alternativas energéticas más limpias.",
    "La protección de los manglares es esencial para numerosas especies marinas.",
    "Los sistemas de inteligencia artificial están optimizando el tráfico en las ciudades.",
    "La conservación de la biodiversidad es fundamental para el equilibrio ecológico."

]

labels = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
          0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]  # 0 para Tecnología, 1 para Naturaleza

# Dividir en conjuntos de entrenamiento y validación
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.1)

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
train_dataset = TextDataset(tokenizer, train_texts, train_labels) # Es el conjunto que usamos paa ir entrenando
val_dataset = TextDataset(tokenizer, val_texts, val_labels)       # El conjunto con el que validamos si las clasficiaciones se van haciendo bien

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2)

# Crear el modelo
num_labels = 2  # Cambia esto según el número de categorías que tengas
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=num_labels)

# Optimizador
optimizer = AdamW(model.parameters(), lr=2e-5)

# Calcula la precisión en las estimaciones
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten() # Nos quedamos con la prediccion de mayor probabildiad
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat) ## Ver si la predicción es la que debería haber sido... Y calculo la media

# Entrenamiento y validación
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(30):  # Número de épocas
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        model.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Train loss: {avg_train_loss}")

    # Validación
    model.eval()
    total_eval_accuracy = 0
    for batch in val_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        with torch.no_grad(): # Sin calculo de gradientes, ya que estos datos no van a usarse para entrenar / recalcular pesos.
            #Simplemente acelera el proceso y hace que consuma menos memoria
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print(f"Epoch {epoch + 1}, Validation Accuracy: {avg_val_accuracy}")

# Función para usar el modelo
def classify_text(text):
    model.eval()
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors="pt", max_length=512, truncation=True, padding='max_length')
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    logits = outputs.logits # El modelo me da las probabildiades de asignarlo a cada categoría
    prediction = torch.argmax(logits, dim=1).cpu().numpy()[0] # Me quedo con la categoría de más alta probabildiad
    return prediction

# Uso del modelo para clasificar un texto
test_text = "La inteligencia artificial está de moda"
print(classify_text(test_text))
test_text = "El océano Pacífico es muy grande"
print(classify_text(test_text))


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Train loss: 0.6105198431760073
Epoch 1, Validation Accuracy: 1.0
Epoch 2, Train loss: 0.18273672135546803
Epoch 2, Validation Accuracy: 1.0
Epoch 3, Train loss: 0.05561182420933619
Epoch 3, Validation Accuracy: 1.0
Epoch 4, Train loss: 0.011081409669714049
Epoch 4, Validation Accuracy: 1.0
Epoch 5, Train loss: 0.00657506231800653
Epoch 5, Validation Accuracy: 1.0
Epoch 6, Train loss: 0.004211019113427028
Epoch 6, Validation Accuracy: 1.0
Epoch 7, Train loss: 0.003132553320028819
Epoch 7, Validation Accuracy: 1.0
Epoch 8, Train loss: 0.0026367299651610665
Epoch 8, Validation Accuracy: 1.0
Epoch 9, Train loss: 0.0021701109362766147
Epoch 9, Validation Accuracy: 1.0
Epoch 10, Train loss: 0.0017796999381971546
Epoch 10, Validation Accuracy: 1.0
Epoch 11, Train loss: 0.0015231333345582243
Epoch 11, Validation Accuracy: 1.0
Epoch 12, Train loss: 0.0013576272885984508
Epoch 12, Validation Accuracy: 1.0
Epoch 13, Train loss: 0.0012185384548502043
Epoch 13, Validation Accuracy: 1.0
Epo

In [6]:
test_text = "El océano Pacífico no es un lago artificial"
print(classify_text(test_text))


1
